In [13]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import os

In [14]:
# Path to your CSVs
path = "/Users/harshit/Downloads/MVE/dataframes/*.csv"
csv_files = glob.glob(path)

print(f"Found {len(csv_files)} CSV files.\n")

# Print filename and number of rows
for file in csv_files:
    try:
        df = pd.read_csv(file)
        token = os.path.basename(file).replace(".csv", "")
        print(f"{token}: {df.shape[0]} rows")
    except Exception as e:
        print(f"Failed to read {file}: {e}")

Found 4 CSV files.

XPARTY: 21 rows
PVE: 215 rows
TWSC: 53 rows
DOGE: 12 rows


In [15]:
import pandas as pd
import glob
import os

path = "/Users/harshit/Downloads/MVE/dataframes/*.csv"
csv_files = glob.glob(path)

dfs = {}
for file in csv_files:
    token = os.path.basename(file).replace(".csv", "")
    df = pd.read_csv(file)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    dfs[token] = df

print(f"Loaded tokens: {list(dfs.keys())}")


Loaded tokens: ['XPARTY', 'PVE', 'TWSC', 'DOGE']


### If Invested 1000usd 20-hrs ago

In [16]:
import numpy as np

# Investment amount
total_investment = 1000
tokens = list(dfs.keys())
num_tokens = len(tokens)
investment_per_token = total_investment / num_tokens

# Find the start time (20 hours ago from the last timestamp in data)
latest_time = min(df.index.max() for df in dfs.values())  # safest: minimum latest time among tokens
start_time = latest_time - pd.Timedelta(hours=20)

print(f"Latest time in data: {latest_time}")
print(f"Start time (20 hrs ago): {start_time}")

portfolio_value = 0

for token, df in dfs.items():
    # Find closest timestamps
    start_price_idx = df.index.get_indexer([start_time], method='nearest')[0]
    start_price = df.iloc[start_price_idx]['close']
    
    end_price = df['close'].iloc[-1]
    
    # Calculate number of tokens bought at start_price
    tokens_bought = investment_per_token / start_price
    
    # Current value of tokens
    current_value = tokens_bought * end_price
    
    print(f"{token} | Buy Price: {start_price:.6f} | End Price: {end_price:.6f} | Current Value: ${current_value:.2f}")
    
    portfolio_value += current_value

print(f"\nTotal portfolio value now: ${portfolio_value:.2f}")
print(f"Total return: {(portfolio_value / total_investment - 1) * 100:.2f}%")


Latest time in data: 2025-06-06 00:00:00+00:00
Start time (20 hrs ago): 2025-06-05 04:00:00+00:00
XPARTY | Buy Price: 0.000331 | End Price: 0.000331 | Current Value: $250.00
PVE | Buy Price: 0.000004 | End Price: 0.000067 | Current Value: $4041.90
TWSC | Buy Price: 0.000137 | End Price: 0.000243 | Current Value: $444.32
DOGE | Buy Price: 0.000152 | End Price: 0.000152 | Current Value: $250.00

Total portfolio value now: $4986.21
Total return: 398.62%


### If invested 1000usd 5-hrs ago

In [17]:
from datetime import timedelta

# Latest timestamp in the data (assuming all dfs have same latest index)
latest_time = max(df.index.max() for df in dfs.values())

# Buy time = 5 hours before latest timestamp
buy_time = latest_time - timedelta(hours=5)

total_investment = 1000
tokens = list(dfs.keys())
num_tokens = len(tokens)
investment_per_token = total_investment / num_tokens

portfolio_value = 0
print(f"Latest time in data: {latest_time}")
print(f"Start time (5 hrs ago): {buy_time}")

for token in tokens:
    df = dfs[token]

    pos = df.index.get_indexer([buy_time], method='nearest')[0]
    buy_price = df.iloc[pos]['close']
    end_price = df['close'].iloc[-1]

    quantity = investment_per_token / buy_price
    current_value = quantity * end_price

    print(f"{token} | Buy Price: {buy_price:.6f} | End Price: {end_price:.6f} | Current Value: ${current_value:.2f}")

    portfolio_value += current_value

total_return = (portfolio_value / total_investment - 1) * 100
print(f"\nTotal portfolio value now: ${portfolio_value:.2f}")
print(f"Total return: {total_return:.2f}%")


Latest time in data: 2025-06-06 09:30:00+00:00
Start time (5 hrs ago): 2025-06-06 04:30:00+00:00
XPARTY | Buy Price: 0.000501 | End Price: 0.000331 | Current Value: $165.11
PVE | Buy Price: 0.001379 | End Price: 0.000067 | Current Value: $12.14
TWSC | Buy Price: 0.000778 | End Price: 0.000243 | Current Value: $78.14
DOGE | Buy Price: 0.000343 | End Price: 0.000152 | Current Value: $111.11

Total portfolio value now: $366.49
Total return: -63.35%
